In [1]:
from torchvision import transforms
from torch.utils import data
from tqdm import tqdm
import os
from vid_dataset_framed import *
from tvn1 import *

/home/ichuviliaeva/miniconda3/envs/hw_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vid_transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((200, 200)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
gpu = torch.device('cuda:0')

In [5]:
torch.cuda.empty_cache()

In [6]:
train_dataset = VideoDataset('/DATA/ichuviliaeva/videos/UCF50/', 'train_index.txt', vid_transforms)

val_dataset = VideoDataset('/DATA/ichuviliaeva/videos/UCF50/', 'test_index.txt', vid_transforms)

In [7]:
train_dataloader = data.DataLoader(train_dataset, batch_size = 4, shuffle = True)
val_dataloader = data.DataLoader(val_dataset, batch_size = 4)

In [8]:
model = TVN1(50).to(gpu)

In [9]:
optimizer = torch.optim.Adam(model.parameters())

In [10]:
criterion = nn.CrossEntropyLoss()

In [11]:
def train(epoch = 1, verbose = 2, model = model, optimazer = optimizer, criterion = criterion, 
          train_dataloader = train_dataloader, val_dataloader = val_dataloader):
    for t in range(epoch):
        loss_list = []
        for x, vid_lens, labels in tqdm(train_dataloader):
            x = torch.stack([x[b, v] for b in range(x.shape[0]) for v in range(x.shape[1])])
            x = x.to(gpu)
            vid_lens = vid_lens.to(gpu)
            labels = labels.to(gpu)
            optimizer.zero_grad()
            res = model((x, vid_lens))
            loss = criterion(res, labels)
            loss_list.append(loss.detach())
            loss.backward()
            optimizer.step()

        print('epoch ', t, ':')
        print('mean loss = ', torch.mean(torch.tensor(loss_list)))
            
        if t % verbose == 0 or t == epoch - 1:
            mi = 0
            acc = 0
            with torch.no_grad():
                loss_val_list = []
                for x, vid_lens, labels in tqdm(val_dataloader):
                    x = torch.stack([x[b, v] for b in range(x.shape[0]) for v in range(x.shape[1])])
                    x = x.to(gpu)
                    vid_lens = vid_lens.to(gpu)
                    labels = labels.to(gpu)
                    predicts = model((x, vid_lens))
                    loss = criterion(predicts, labels)
                    loss_val_list.append(loss.detach())
                    acc += torch.sum(torch.eq(torch.argmax(predicts, dim=-1), labels).to(dtype=torch.float64)).item()
                    mi += x.shape[0]
                print('mean val loss = ', torch.mean(torch.tensor(loss_val_list)))
                print('accuracy = ', acc / (mi + (mi == 0)))
                
                if t % verbose % 2 == 0 or t == epoch - 1:
                    torch.save(model.state_dict(), 'tvn1-epoch-' + str(t) + '-framed.pth')

In [ ]:
train(epoch = 40)

100%|███████████████████████████████████████| 1420/1420 [13:20<00:00,  1.77it/s]


epoch  0 :
mean loss =  tensor(3.9117)


100%|█████████████████████████████████████████| 251/251 [01:55<00:00,  2.18it/s]


mean val loss =  tensor(3.9076)
accuracy =  0.0037387836490528413


100%|███████████████████████████████████████| 1420/1420 [15:08<00:00,  1.56it/s]


...

100%|█████████████████████████████████████████| 251/251 [01:44<00:00,  2.39it/s]


mean val loss =  tensor(3.9056)
accuracy =  0.0037387836490528413


100%|███████████████████████████████████████| 1420/1420 [14:31<00:00,  1.63it/s]


epoch  11 :
mean loss =  tensor(3.9048)


 47%|██████████████████▋                     | 662/1420 [06:45<07:35,  1.66it/s]